In [5]:
import os
import numpy as np
import cv2

# Paths
padded_cusp_image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/padded/images"
non_cusp_image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/non_cusp_images"

padded_mask_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/padded/masks"
# Image settings
IMG_SIZE = 224  # U-Net or ResNet input size (same size)

# Load and preprocess images
def load_images(image_dir, mask_dir=None, label=1):
    images, masks, labels = [], [], []

    for filename in os.listdir(image_dir):
        img_path = os.path.join(image_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"Skipping {filename} (missing image)")
            continue

        # Resize to 224x224
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        # Normalize to [0,1]
        img = img.astype(np.float32) / 255.0
        img = np.expand_dims(img, axis=-1)  # Add channel dim for CNN (e.g., (224, 224, 1))

        images.append(img)

        # If mask exists, load and process it
        if mask_dir:
            mask_path = os.path.join(mask_dir, filename)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            if mask is not None:
                mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))  # Resize to match image size
                mask = mask.astype(np.float32) / 255.0  # Normalize to [0,1]
                mask = np.expand_dims(mask, axis=-1)  # Add channel dim (e.g., (224, 224, 1))
                masks.append(mask)
            else:
                print(f"Skipping {filename} (missing mask)")

        labels.append(label)

    return np.array(images), np.array(masks) if mask_dir else None, np.array(labels)

# Load cusp images & masks
cusp_images, cusp_masks, cusp_labels = load_images(padded_cusp_image_dir, padded_mask_dir, label=1)

# Load non-cusp images (no masks)
non_cusp_images, _, non_cusp_labels = load_images(non_cusp_image_dir, mask_dir=None, label=0)

# Combine datasets
X = np.concatenate((cusp_images, non_cusp_images), axis=0)
y = np.concatenate((cusp_masks, non_cusp_images), axis=0)  # The labels are now the segmentation masks

# Shuffle dataset
indices = np.arange(len(X))
np.random.shuffle(indices)
X, y = X[indices], y[indices]

# Now, X contains image data and y contains the segmentation mask labels



In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

IMG_SIZE = 224  # Input size for ResNet
NUM_CLASSES = 1  # Binary classification: cusp or no cusp

def unet_model(input_size=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = Input(input_size)

    # Contracting path
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)

    # Bottom part (bottleneck)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)

    # Expansive path (upscaling)
    up1 = UpSampling2D((2, 2))(conv3)
    concat1 = Concatenate()([up1, conv2])
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat1)

    up2 = UpSampling2D((2, 2))(conv4)
    concat2 = Concatenate()([up2, conv1])
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat2)

    # Final layer: Sigmoid for binary segmentation
    output = Conv2D(NUM_CLASSES, (1, 1), activation='sigmoid')(conv5)

    model = Model(inputs, output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model summary
model = unet_model()
history = model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)
# Assuming you've already prepared your images (X) and masks (Y)




Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 224, 224, 1)